# Python training for data engineers
## 05. Data cleaning
In the first example we will deal with the XML response of the first request. Import lxml to handle the XML response we got in the web crawling step.

Make sure `lxml` is installed, or run `pip install lxml` first to install the library.

In [1]:
# Load data from previous step
import pickle
xmlcontent = pickle.load(open("xmlcontent_notebook_04.pickle", "rb"))

In [2]:
import lxml.html

and construct the [XML tree](https://www.w3schools.com/xml/xml_tree.asp)

In [3]:
xmltree = lxml.html.fromstring(xmlcontent)
xmltree

<Element html at 0x7f760989f788>

The HTML source for the result page is shown in the following snippet.
```html
<!DOCTYPE html>
<html>
<head>
  <title></title>
</head>
<body>
  <table class="list">
    <tbody>
      <tr>
        <th>Package</th>
        <th><u title="Occurrence of search term weighted by field (name, summary, keywords, description, author, maintainer)">Weight*</u></th>
        <th>Description</th>
      </tr>
      <tr class="odd">
        <td>
          <a href="/pypi/machineLearningStanford/0.0">machineLearningStanford&nbsp;0.0</a>
        </td>
        <td>14</td>
        <td>Machine Learning Stanford</td>
      </tr>
      <tr class="even">
        <td>
          <a href="/pypi/pylearning/3.2.2b1">pylearning&nbsp;3.2.2b1</a>
        </td>
        <td>14</td>
        <td>Simple high-level library to use machine learning algorithms</td>
      </tr>
    </tbody>
  </table>
</body>
</html>
```

Using the `tag` command we can see the top element of the XML tree, which indeed is the first element as we expect from the XML above:

In [4]:
xmltree.tag

'html'

We want to get all links from the table and add them to a list.

```html
<table class="list"><td><a href="link">
```

Make sure `cssselect` is installed inside the environment.

In [6]:
for link in xmltree.cssselect('table[class*=\'list\'] td a'):
    print(link.get('href'))

/pypi/watson-machine-learning-client/1.0.83
/pypi/azure-mgmt-machinelearningcompute/0.4.0
/pypi/machineLearningStanford/0.0
/pypi/pylearning/3.2.2b1
/pypi/Etherscan-Magic-for-Machine-Learning-and-Bash/0.1
/pypi/oolearning/0.1.19
/pypi/Augmentor/0.2.0
/pypi/auto_ml/2.9.10
/pypi/automl/2.9.9
/pypi/azure-ml-api-sdk/0.1.0a11
/pypi/azureml-model-management-sdk/1.0.1b6
/pypi/azureml.datacollector/0.1.0a13
/pypi/biovida/0.1.1
/pypi/candle/0.0.1
/pypi/chemml/0.4.2
/pypi/ClassificaIO/1.0.5
/pypi/concordia/0.1.1
/pypi/costcla/0.5
/pypi/cxflow/0.11.2
/pypi/deeplearning/0.0.1
/pypi/django-estimators/0.2.1
/pypi/dlearn/0.1.0
/pypi/dopaminekit/0.9.3
/pypi/elm/0.1.1
/pypi/epysteme/0.1.0
/pypi/estimators/0.1.0.dev0
/pypi/featureforge/0.1.6
/pypi/FukuML/0.4.1
/pypi/genetics/1.0.0
/pypi/h2o/3.18.0.5
/pypi/h2o-pysparkling-2.3/2.3.0
/pypi/h2o_pysparkling_1.6/1.6.13
/pypi/h2o_pysparkling_2.0/2.0.26
/pypi/h2o_pysparkling_2.1/2.1.25
/pypi/h2o_pysparkling_2.2/2.2.11
/pypi/happyml/0.0.5
/pypi/healthcareai/1.0


/pypi/MKLpy/0.2.1b0
/pypi/ml/0.0.0
/pypi/ml-lib/0.0.3
/pypi/ml-toolkit/0.07
/pypi/ml-tools/0.0.15
/pypi/ml_metrics/0.1.4
/pypi/mlboost/0.4.1
/pypi/mlbox/0.5.3
/pypi/mlconf/0.0.3
/pypi/MLDateTimeParser/0.0.0
/pypi/mlfromscratch/0.0.4
/pypi/mlkit/0.0.1
/pypi/mlperceptron/0.3
/pypi/mlpipes/0.1
/pypi/mlPyUtils/0.1
/pypi/mlstatpy/0.1.335
/pypi/mltk/0.0.5
/pypi/mlutility/0.1.1
/pypi/mlxtend/0.11.0
/pypi/mmbot/1.0.10
/pypi/modelchimp/0.2.1
/pypi/modelvis/0.1.2
/pypi/modlamp/3.4.0
/pypi/mondrianforest/0.0.2
/pypi/monkeylearn/0.3.7
/pypi/moodleinspire/0.0.9
/pypi/mooncake_utils/0.5
/pypi/moxel/0.0.3.post41
/pypi/msm-design/0.12
/pypi/multilabel-metrics/0.0.1
/pypi/musicmood/0.1.2
/pypi/mxboard/0.1.0rc9
/pypi/mxnet-to-coreml/0.1.2
/pypi/NaiveBayes/1.0.0
/pypi/neatbook/0.20
/pypi/neokami-sdk/0.2
/pypi/nereval/0.2.4
/pypi/NetBuilder/0.2.2
/pypi/netception/0.2.0
/pypi/networks/0.3.7
/pypi/neuralcli/1.6
/pypi/nlup/0.5
/pypi/nn/0.0.2
/pypi/nnabla/0.9.9
/pypi/nnabla_ext-cuda/0.9.9
/pypi/NNBuilder/0.3.

Lets save them to an array:

In [7]:
# Create an empty list
list_of_links = []
# Loop through all the links in the table
for link in xmltree.cssselect('table[class*=\'list\'] td a'):
    # Prepend with full URL and remove the version tag at the end by splitting by '/' and taking the relevant parts
    list_of_links.append('https://pypi.python.org' + link.get('href').rsplit('/',1)[0])

Show the first element of the list.

In [8]:
list_of_links[0]

'https://pypi.python.org/pypi/watson-machine-learning-client'

#### Getting detailed information per package
Next we will loop over the links we retrieved in the previous step. Per detail page we will extract information about the packages.

Example URL: https://pypi.python.org/pypi/scikit-learn/0.19.0

```html
<table class="list" style="margin-bottom: 10px;">
<tbody><tr>
  <th>File</th>
  <th>Type</th>
  <th>Py Version</th>
  <th>Uploaded on</th>
  <th style="text-align: right;">Size</th>
</tr>
<tr class="odd">
    <td>
    <span style="white-space: nowrap;">
      <a href="https://pypi.python.org/packages/a2/4d/9dc527caffc558a225a481e74518be9d0812c3ee8b4e1b12e16d8fec8d65/machineLearningStanford-0.0.tar.gz#md5=b71099d4e405f7888d75ffcf8f71fce0">machineLearningStanford-0.0.tar.gz</a>
      (<a title="MD5 Digest" href="/pypi?:action=show_md5&amp;digest=b71099d4e405f7888d75ffcf8f71fce0">md5</a>)
    </span>
  </td>
  <td style="white-space: nowrap;">
    Source
  </td>
  <td>
  </td>
  <td>2015-07-08</td>
  <td style="text-align: right;">3KB</td>
</tr>
<tr><td id="last" colspan="6"></td></tr>
  </tbody></table>
```

Retrieve information for the first link by requesting the URL and convert it to an XML tree.

In [9]:
import requests
response = requests.get(list_of_links[0])
xmltree = lxml.html.fromstring(response.content)

Next we will go through the table as depicted in the HTML snippet above, and extract the relevant information.


In [10]:
# Create dictionary
module_info = {}
for entry in xmltree.cssselect('table[class*=\'list\'] tr')[1:-1]:
    module_info['filename'] = entry.cssselect('td')[0].text_content()
    module_info['release_type'] = entry.cssselect('td')[1].text_content()
    module_info['python_version'] = entry.cssselect('td')[2].text_content()
    module_info['uploaded_on'] = entry.cssselect('td')[3].text_content()
    module_info['size'] = entry.cssselect('td')[4].text_content()
module_info

{'filename': '\n    \n      watson_machine_learning_client-1.0.83.tar.gz\n      (md5)\n    \n    \n        ',
 'python_version': '\n    \n  ',
 'release_type': '\n    Source\n  ',
 'size': '211KB',
 'uploaded_on': '2018-04-10'}

As we can see in the `module_info` different values have weird characters. We need to remove spaces and new line characters.

In [11]:
module_info = {}
for entry in xmltree.cssselect('table[class*=\'list\'] tr')[1:-1]:
    module_info['filename'] = entry.cssselect('td')[0].text_content().strip().replace('\n', '')
    module_info['release_type'] = entry.cssselect('td')[1].text_content().strip().replace('\n', '').lower()
    module_info['python_version'] = entry.cssselect('td')[2].text_content().strip().replace('\n', '')
    module_info['uploaded_on'] = entry.cssselect('td')[3].text_content()
    module_info['size'] = entry.cssselect('td')[4].text_content()
module_info

{'filename': 'watson_machine_learning_client-1.0.83.tar.gz      (md5)',
 'python_version': '',
 'release_type': 'source',
 'size': '211KB',
 'uploaded_on': '2018-04-10'}

Lets put this in a function, so we can simply extract the data by calling the function with the URL's for the packages.

In [12]:
def get_info_from_link(url):
    response = requests.get(url)
    xmltree = lxml.html.fromstring(response.content)
    module_info_list = []
    for entry in xmltree.cssselect('table[class*=\'list\'] tr')[1:-1]:
        module_info = {}
        module_info['package_name'] = url.rsplit('/',1)[1]
        module_info['filename'] = entry.cssselect('td')[0].text_content().strip().replace('\n', '')
        module_info['release_type'] = entry.cssselect('td')[1].text_content().strip().replace('\n', '').lower()
        try:
            module_info['python_version'] = entry.cssselect('td')[2].text_content().strip().replace('\n', '')
        except:
            pass # No version found
        try:
            module_info['uploaded_on'] = entry.cssselect('td')[3].text_content()
        except:
            pass # No date found
        try:
            module_info['size'] = entry.cssselect('td')[4].text_content()
        except:
            pass # No size found
        module_info_list.append(module_info)
    return module_info_list

Execute the function for the first link.

In [13]:
info = get_info_from_link(list_of_links[0])
info

[{'filename': 'watson_machine_learning_client-1.0.83-py3-none-any.whl      (md5)',
  'package_name': 'watson-machine-learning-client',
  'python_version': 'py3',
  'release_type': 'python wheel',
  'size': '552KB',
  'uploaded_on': '2018-04-10'},
 {'filename': 'watson_machine_learning_client-1.0.83.tar.gz      (md5)',
  'package_name': 'watson-machine-learning-client',
  'python_version': '',
  'release_type': 'source',
  'size': '211KB',
  'uploaded_on': '2018-04-10'}]

From the info shown above, we can conclude that there are different versions for the module available.

Lets extract data for the second link.

In [14]:
info_two = get_info_from_link(list_of_links[1])
info_two

[{'filename': 'azure-mgmt-machinelearningcompute-0.4.0.zip      (md5)',
  'package_name': 'azure-mgmt-machinelearningcompute',
  'python_version': '',
  'release_type': 'source',
  'size': '50KB',
  'uploaded_on': '2018-01-02'},
 {'filename': 'azure_mgmt_machinelearningcompute-0.4.0-py2.py3-none-any.whl      (md5)',
  'package_name': 'azure-mgmt-machinelearningcompute',
  'python_version': 'py2.py3',
  'release_type': 'python wheel',
  'size': '38KB',
  'uploaded_on': '2018-01-02'}]

We can now combine the two information blocks into one list by creating a new list.

In [15]:
total_info = [info, info_two]
total_info

[[{'filename': 'watson_machine_learning_client-1.0.83-py3-none-any.whl      (md5)',
   'package_name': 'watson-machine-learning-client',
   'python_version': 'py3',
   'release_type': 'python wheel',
   'size': '552KB',
   'uploaded_on': '2018-04-10'},
  {'filename': 'watson_machine_learning_client-1.0.83.tar.gz      (md5)',
   'package_name': 'watson-machine-learning-client',
   'python_version': '',
   'release_type': 'source',
   'size': '211KB',
   'uploaded_on': '2018-04-10'}],
 [{'filename': 'azure-mgmt-machinelearningcompute-0.4.0.zip      (md5)',
   'package_name': 'azure-mgmt-machinelearningcompute',
   'python_version': '',
   'release_type': 'source',
   'size': '50KB',
   'uploaded_on': '2018-01-02'},
  {'filename': 'azure_mgmt_machinelearningcompute-0.4.0-py2.py3-none-any.whl      (md5)',
   'package_name': 'azure-mgmt-machinelearningcompute',
   'python_version': 'py2.py3',
   'release_type': 'python wheel',
   'size': '38KB',
   'uploaded_on': '2018-01-02'}]]

Lets extract the information for <u>all</u> the links to make a more interesting dataset.

In [16]:
# Initialize an empty list
all_info = []
for link in list_of_links:
    # Extract the data
    info_list = get_info_from_link(link)
    # Append the info to the big list
    all_info += info_list
all_info

[{'filename': 'watson_machine_learning_client-1.0.83-py3-none-any.whl      (md5)',
  'package_name': 'watson-machine-learning-client',
  'python_version': 'py3',
  'release_type': 'python wheel',
  'size': '552KB',
  'uploaded_on': '2018-04-10'},
 {'filename': 'watson_machine_learning_client-1.0.83.tar.gz      (md5)',
  'package_name': 'watson-machine-learning-client',
  'python_version': '',
  'release_type': 'source',
  'size': '211KB',
  'uploaded_on': '2018-04-10'},
 {'filename': 'azure-mgmt-machinelearningcompute-0.4.0.zip      (md5)',
  'package_name': 'azure-mgmt-machinelearningcompute',
  'python_version': '',
  'release_type': 'source',
  'size': '50KB',
  'uploaded_on': '2018-01-02'},
 {'filename': 'azure_mgmt_machinelearningcompute-0.4.0-py2.py3-none-any.whl      (md5)',
  'package_name': 'azure-mgmt-machinelearningcompute',
  'python_version': 'py2.py3',
  'release_type': 'python wheel',
  'size': '38KB',
  'uploaded_on': '2018-01-02'},
 {'filename': 'machineLearningStanfor

### Data conversion
Import pandas so we can start using dataframes with our retrieved data.

In [17]:
import pandas as pd

In [18]:
xmldf = pd.DataFrame.from_dict(all_info)
xmldf.head()

,filename,package_name,python_version,release_type,size,uploaded_on
0,watson_machine_learning_client-1.0.83-py3-none...,watson-machine-learning-client,py3,python wheel,552KB,2018-04-10
1,watson_machine_learning_client-1.0.83.tar.gz ...,watson-machine-learning-client,,source,211KB,2018-04-10
2,azure-mgmt-machinelearningcompute-0.4.0.zip ...,azure-mgmt-machinelearningcompute,,source,50KB,2018-01-02
3,azure_mgmt_machinelearningcompute-0.4.0-py2.py...,azure-mgmt-machinelearningcompute,py2.py3,python wheel,38KB,2018-01-02
4,machineLearningStanford-0.0.tar.gz (md5),machineLearningStanford,,source,3KB,2015-07-08


Show the datatypes of the dataframe.

In [19]:
xmldf.dtypes

filename          object
package_name      object
python_version    object
release_type      object
size              object
uploaded_on       object
dtype: object

Convert the `release_type` to a category.

In [20]:
xmldf['release_type'] = xmldf['release_type'].astype('category')

Convert the `uploaded_on` to a proper timestamp using `pd.to_datetime()`.

In [21]:
xmldf['uploaded_on'] = pd.to_datetime(xmldf['uploaded_on'])

Split the `size` into a `size` and a `unit` by using a regular expression.

In [22]:
xmldf[['size', 'unit']] = xmldf['size'].str.extract('(\d*)(\w*?)$', expand=True)

Fill the empty `size` values with a 0.

In [23]:
xmldf['size'] = xmldf['size'].fillna(0).astype('int')

Check the datatypes again.

In [24]:
xmldf.dtypes

filename                  object
package_name              object
python_version            object
release_type            category
size                       int64
uploaded_on       datetime64[ns]
unit                      object
dtype: object

### Lambda function
Define a function to convert the size to bytes based on the unit.

In [25]:
def convert_to_bytes(size, unit):
    if unit == 'KB':
        size = size*1024
    elif unit == 'MB':
        size = size*1024*1024
    else:
        size = size
    return size

Apply the function to convert the sizes using the `lambda` function.

In [26]:
xmldf['size_in_bytes'] = xmldf.apply(lambda row: convert_to_bytes(row['size'], row['unit']), axis=1)
xmldf.head()

,filename,package_name,python_version,release_type,size,uploaded_on,unit,size_in_bytes
0,watson_machine_learning_client-1.0.83-py3-none...,watson-machine-learning-client,py3,python wheel,552,2018-04-10,KB,565248
1,watson_machine_learning_client-1.0.83.tar.gz ...,watson-machine-learning-client,,source,211,2018-04-10,KB,216064
2,azure-mgmt-machinelearningcompute-0.4.0.zip ...,azure-mgmt-machinelearningcompute,,source,50,2018-01-02,KB,51200
3,azure_mgmt_machinelearningcompute-0.4.0-py2.py...,azure-mgmt-machinelearningcompute,py2.py3,python wheel,38,2018-01-02,KB,38912
4,machineLearningStanford-0.0.tar.gz (md5),machineLearningStanford,,source,3,2015-07-08,KB,3072


In [27]:
xmldf.dtypes

filename                  object
package_name              object
python_version            object
release_type            category
size                       int64
uploaded_on       datetime64[ns]
unit                      object
size_in_bytes              int64
dtype: object

In [28]:
xmldf.to_pickle('xml_dataframe_notebook_05.pickle')

## Handling the JSON data
Lets read the JSON data into a dataframe via a pickle:

In [29]:
jsoncontent = pickle.load(open("jsoncontent_notebook_04.pickle", "rb"))

In [30]:
jsoncontent

{u'has_more': False,
 u'items': [{u'count': 933520,
   u'has_synonyms': True,
   u'is_moderator_only': False,
   u'is_required': False,
   u'name': u'python'},
  {u'count': 86425,
   u'has_synonyms': False,
   u'is_moderator_only': False,
   u'is_required': False,
   u'name': u'django'},
  {u'count': 61821,
   u'has_synonyms': True,
   u'is_moderator_only': False,
   u'is_required': False,
   u'name': u'python-3.x'},
  {u'count': 58237,
   u'has_synonyms': False,
   u'is_moderator_only': False,
   u'is_required': False,
   u'name': u'pandas'},
  {u'count': 52986,
   u'has_synonyms': False,
   u'is_moderator_only': False,
   u'is_required': False,
   u'name': u'python-2.7'},
  {u'count': 42194,
   u'has_synonyms': False,
   u'is_moderator_only': False,
   u'is_required': False,
   u'name': u'numpy'},
  {u'count': 30260,
   u'has_synonyms': True,
   u'is_moderator_only': False,
   u'is_required': False,
   u'name': u'list'},
  {u'count': 25618,
   u'has_synonyms': True,
   u'is_moderator

We need to convert the list of items to a dataframe:

In [31]:
jsondf = pd.DataFrame.from_dict(jsoncontent['items'])
jsondf.head()

,count,has_synonyms,is_moderator_only,is_required,name
0,933520,True,False,False,python
1,86425,False,False,False,django
2,61821,True,False,False,python-3.x
3,58237,False,False,False,pandas
4,52986,False,False,False,python-2.7


We can observe that the datatypes of the dataframe are inherited, in contrast to the XML experiment from the first section of this notebook.

In [32]:
jsondf.dtypes

count                 int64
has_synonyms           bool
is_moderator_only      bool
is_required            bool
name                 object
dtype: object

Lets extract all the Python related tags from the response.

In [33]:
DATA = {}
# Iterate over all the items
for item in jsoncontent['items']:
    # Check if python is absent in the DATA dictionary
    if 'python' not in DATA.keys():
        # Create a new empty dictionary for the python key in DATA
        DATA['python'] = {}
    # Add all non-python items to the python key
    if item['name'] != 'python':
        # Create a key in the python dictionary with the number of references
        DATA['python'][item['name']] = item['count']

Show the first tag.

In [34]:
for key in DATA['python'].keys():
    print(key)
    break

selenium


The above key is one of the many items related to Python according to the StackOverflow API. Lets use this key to extract all tags related to the key.

In [35]:
API_URL = 'https://api.stackexchange.com/2.2/tags/%s/related?pagesize=100&site=stackoverflow' % key

Retrieve the related tags for the first key found above.

In [36]:
response = requests.get(API_URL)
data = response.json()

for item in data['items']:
    if key not in DATA.keys():
        DATA[key] = {}
    if item['name'] != key:
        DATA[key][item['name']] = item['count']

Current dataframe:

In [37]:
pd.DataFrame.from_dict(DATA, orient='index')

,selenium,linux,subprocess,dataframe,datetime,pip,file,mysql,loops,sockets,...,selenium-ide,ruby-on-rails,eclipse,robotframework,testng,selenium-grid,watir,selenium-chromedriver,selenium-rc,css-selectors
python,10473.0,10064.0,6168.0,14470.0,6549.0,6879.0,6687.0,10599.0,7265.0,6224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
selenium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1156.0,478.0,799.0,470.0,1439.0,726.0,367.0,2623.0,880.0,385.0


As we can observe we are creating a 2 dimensional matrix with a count between the horizontal and vertical item.

Show the existing keys:

In [38]:
print(DATA.keys())

['python', u'selenium']


Lets create a function for the code we just ran.

In [39]:
def add_items_from_key(key):
    API_URL = 'https://api.stackexchange.com/2.2/tags/%s/related?pagesize=100&site=stackoverflow' % key
    response = requests.get(API_URL)
    data = response.json()

    for item in data['items']:
        if key not in DATA.keys():
            DATA[key] = {}
        if item['name'] != key:
            DATA[key][item['name']] = item['count']

Now iterate over the keys for Python and retrieve all the items per key.

In [40]:
for key in DATA['python'].keys():
    if (key not in DATA.keys()):
        add_items_from_key(key)

Create the final dataframe.

In [41]:
df = pd.DataFrame.from_dict(DATA, orient='index')
df.head()

,selenium-webdriver,appium,firefox,python-2.7,testing,node.js,phpunit,angularjs,nightwatch.js,selenium-firefoxdriver,...,model,django-managers,django-validation,django-serializer,django-1.7,django-testing,django-users,django-database,django-migrations,django-signals
algorithm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
arrays,NaN,NaN,NaN,NaN,NaN,2339.0,NaN,3426.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
beautifulsoup,83.0,NaN,NaN,1186.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
c++,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
class,NaN,NaN,NaN,572.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Finally, replace all NaNs with zero:

In [42]:
df = df.fillna(0)

In [43]:
df.head()

,selenium-webdriver,appium,firefox,python-2.7,testing,node.js,phpunit,angularjs,nightwatch.js,selenium-firefoxdriver,...,model,django-managers,django-validation,django-serializer,django-1.7,django-testing,django-users,django-database,django-migrations,django-signals
algorithm,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
arrays,0.0,0.0,0.0,0.0,0.0,2339.0,0.0,3426.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
beautifulsoup,83.0,0.0,0.0,1186.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
c++,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
class,0.0,0.0,0.0,572.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
df.to_pickle("json_dataframe_notebook_05.pickle")